# ADS 509 Module 3: Group Comparison 

The task of comparing two groups of text is fundamental to textual analysis. There are innumerable applications: survey respondents from different segments of customers, speeches by different political parties, words used in Tweets by different constituencies, etc. In this assignment you will build code to effect comparisons between groups of text data, using the ideas learned in reading and lecture.

This assignment asks you to analyze the lyrics for the two artists you selected in Module 1 and the Twitter descriptions pulled for Robyn and Cher. If the results from that pull were not to your liking, you are welcome to use the zipped data from the “Assignment Materials” section. Specifically, you are asked to do the following: 

* Read in the data, normalize the text, and tokenize it. When you tokenize your Twitter descriptions, keep hashtags and emojis in your token set. 
* Calculate descriptive statistics on the two sets of lyrics and compare the results. 
* For each of the four corpora, find the words that are unique to that corpus. 
* Build word clouds for all four corpora. 

Each one of the analyses has a section dedicated to it below. Before beginning the analysis there is a section for you to read in the data and do your cleaning (tokenization and normalization). 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [38]:
import os
import re
import emoji
import pandas as pd
import nltk

from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation
from wordcloud import WordCloud 

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer


In [45]:
# Use this space for any additional import statements you need
import ssl
import nltk

# Create an unverified SSL context
ssl._create_default_https_context = ssl._create_unverified_context

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

import string
from nltk.tokenize import word_tokenize
# Download necessary NLTK data
nltk.download('punkt')      # For tokenization
nltk.download('stopwords')  # For stopwords


[nltk_data] Downloading package punkt to /Users/Nishok/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Nishok/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Nishok/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Nishok/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [47]:
# Place any addtional functions or constants you need here. 

# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

# Stopwords
sw = stopwords.words("english")

# Two useful regex
whitespace_pattern = re.compile(r"\s+")
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")

# It's handy to have a full set of emojis
all_language_emojis = set()

for country in emoji.EMOJI_DATA : 
    for em in emoji.EMOJI_DATA[country] : 
        all_language_emojis.add(em)

# and now our functions
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity, and num_tokens most common
        tokens. Return a list of 
    """

    # Place your Module 2 solution here
    num_tokens = len(tokens)
    num_unique_tokens = len(set(tokens))
    num_characters = len(''.join(tokens))
    lexical_diversity = num_unique_tokens / num_tokens if num_tokens > 0 else 0
    
    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
        print(f"The {num_tokens} most common tokens: {Counter(tokens).most_common(num_tokens)}")
        print(f"The {num_tokens} most common tokens: {Counter(tokens).most_common(num_tokens)}")
    
        # print the five most common tokens
        
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])
    
    return(0)
    
def contains_emoji(s):
    
    s = str(s)
    emojis = [ch for ch in s if emoji.is_emoji(ch)]

    return(len(emojis) > 0)


def remove_stop(tokens) :
    # modify this function to remove stopwords
    return [token for token in tokens if token.lower() not in sw]
 
def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    # modify this function to return tokens
    # Lowercase the text to normalize it
    text = text.lower()
    
    # Split by whitespace
    tokens = re.split(whitespace_pattern, text)
    
    return [token for token in tokens if token]  # Remove empty tokens

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)

## Data Ingestion

Use this section to ingest your data into the data structures you plan to use. Typically this will be a dictionary or a pandas DataFrame.

In [48]:
# Feel fre to use the below cells as an example or read in the data in a way you prefer

data_location = "" # change to your location if it is not in the same directory as your notebook
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

artist_files = {'cher':'cher_followers_data.txt',
                'robyn':'robynkonichiwa_followers_data.txt'}


In [49]:
twitter_data = pd.read_csv(data_location + twitter_folder + artist_files['cher'],
                           sep="\t",
                           quoting=3)

twitter_data['artist'] = "cher"

In [50]:
twitter_data_2 = pd.read_csv(data_location + twitter_folder + artist_files['robyn'],
                             sep="\t",
                             quoting=3)
twitter_data_2['artist'] = "robyn"

twitter_data = pd.concat([
    twitter_data,twitter_data_2])
    
del(twitter_data_2)

In [51]:
cher_lyrics_folder = data_location + lyrics_folder + "cher/"
robyn_lyrics_folder = data_location + lyrics_folder + "robyn/"

def read_lyrics_from_folder(folder_path):
    lyrics = ""
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                lyrics += file.read() + "\n" 
    return lyrics

cher_lyrics = read_lyrics_from_folder(cher_lyrics_folder)
robyn_lyrics = read_lyrics_from_folder(robyn_lyrics_folder)

# Create a DataFrame for lyrics
lyrics_data = pd.DataFrame({
    'artist': ['cher', 'robyn'],
    'lyrics': [cher_lyrics, robyn_lyrics]
})

## Tokenization and Normalization

In this next section, tokenize and normalize your data. We recommend the following cleaning. 

**Lyrics** 

* Remove song titles
* Casefold to lowercase
* Remove stopwords (optional)
* Remove punctuation
* Split on whitespace

Removal of stopwords is up to you. Your descriptive statistic comparison will be different if you include stopwords, though TF-IDF should still find interesting features for you. Note that we remove stopwords before removing punctuation because the stopword set includes punctuation.

**Twitter Descriptions** 

* Casefold to lowercase
* Remove stopwords
* Remove punctuation other than emojis or hashtags
* Split on whitespace

Removing stopwords seems sensible for the Twitter description data. Remember to leave in emojis and hashtags, since you analyze those. 

In [52]:
# Load stopwords (you can download these using nltk.download('stopwords') if not already downloaded)
stop_words = set(stopwords.words('english'))

# Function to remove punctuation (while keeping emojis and hashtags for Twitter)
def remove_punctuation(text, keep_emoji=False):
    if keep_emoji:
        # Removing all punctuation except emojis and hashtags
        return re.sub(r'[^\w\s#]', '', text)
    else:
        # Removing all punctuation
        return text.translate(str.maketrans('', '', string.punctuation))

# Function to tokenize text
def tokenize(text):
    return word_tokenize(text)

# Function to remove stopwords
def remove_stop(tokens):
    return [token for token in tokens if token not in stop_words]

# Function to prepare data through pipeline
def prepare(text, pipeline):
    # Skip processing if the text is NaN or None
    if isinstance(text, str):
        for func in pipeline:
            text = func(text)
        return text
    else:
        return []

# Define the pipeline for processing lyrics and Twitter descriptions
lyrics_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]
twitter_pipeline = [str.lower, lambda text: remove_punctuation(text, keep_emoji=True), tokenize, remove_stop]

# Apply the pipeline to the lyrics data
lyrics_data["tokens"] = lyrics_data["lyrics"].apply(prepare, pipeline=lyrics_pipeline)
lyrics_data["num_tokens"] = lyrics_data["tokens"].map(len)

# Apply the pipeline to the Twitter data
twitter_data["tokens"] = twitter_data["description"].apply(prepare, pipeline=twitter_pipeline)
twitter_data["num_tokens"] = twitter_data["tokens"].map(len)
twitter_data['has_emoji'] = twitter_data["description"].apply(contains_emoji)

# Function to check if the description contains emojis
def contains_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return bool(emoji_pattern.search(text))

# Add a column to Twitter data to flag if it contains emojis
twitter_data['has_emoji'] = twitter_data["description"].apply(contains_emoji)

# Display a sample of 10 Twitter descriptions that contain emojis
emoji_sample = twitter_data[twitter_data['has_emoji']].sample(10)[["description", "tokens"]]

# Output the results
print(emoji_sample)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/Nishok/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [54]:
twitter_data['has_emoji'] = twitter_data["description"].apply(contains_emoji)

Let's take a quick look at some descriptions with emojis.

In [53]:
twitter_data[twitter_data.has_emoji].sample(10)[["artist","description","tokens"]]

AttributeError: 'DataFrame' object has no attribute 'has_emoji'

With the data processed, we can now start work on the assignment questions. 

Q: What is one area of improvement to your tokenization that you could theoretically carry out? (No need to actually do it; let's not make perfect the enemy of good enough.)

A: One area of improvement to my tokenization that I could theoretically carry out

## Calculate descriptive statistics on the two sets of lyrics and compare the results. 


In [ ]:
# your code here


Q: what observations do you make about these data? 

A: 


## Find tokens uniquely related to a corpus

Typically we would use TF-IDF to find unique tokens in documents. Unfortunately, we either have too few documents (if we view each data source as a single document) or too many (if we view each description as a separate document). In the latter case, our problem will be that descriptions tend to be short, so our matrix would be too sparse to support analysis. 

To avoid these problems, we will create a custom statistic to identify words that are uniquely related to each corpus. The idea is to find words that occur often in one corpus and infrequently in the other(s). Since corpora can be of different lengths, we will focus on the _concentration_ of tokens within a corpus. "Concentration" is simply the count of the token divided by the total corpus length. For instance, if a corpus had length 100,000 and a word appeared 1,000 times, then the concentration would be $\frac{1000}{100000} = 0.01$. If the same token had a concentration of $0.005$ in another corpus, then the concentration ratio would be $\frac{0.01}{0.005} = 2$. Very rare words can easily create infinite ratios, so you will also add a cutoff to your code so that a token must appear at least $n$ times for you to return it. 

An example of these calculations can be found in [this spreadsheet](https://docs.google.com/spreadsheets/d/1P87fkyslJhqXFnfYezNYrDrXp_GS8gwSATsZymv-9ms). Please don't hesitate to ask questions if this is confusing. 

In this section find 10 tokens for each of your four corpora that meet the following criteria: 

1. The token appears at least `n` times in all corpora
1. The tokens are in the top 10 for the highest ratio of appearances in a given corpora vs appearances in other corpora.

You will choose a cutoff for yourself based on the side of the corpus you're working with. If you're working with the Robyn-Cher corpora provided, `n=5` seems to perform reasonably well.

In [58]:
# your code here
def descriptive_stats(tokens, num_tokens=5, verbose=True):
    """
    Given a list of tokens, calculate number of tokens, number of unique tokens, 
    number of characters, lexical diversity, and num_tokens most common tokens.
    Return a dictionary of these statistics.
    """
    # Number of tokens
    num_total_tokens = len(tokens)
    
    # Number of unique tokens
    unique_tokens = set(tokens)
    num_unique_tokens = len(unique_tokens)
    
    # Total number of characters (in tokens)
    num_characters = sum(len(token) for token in tokens)
    
    # Lexical diversity: ratio of unique tokens to total tokens
    lexical_diversity = num_unique_tokens / num_total_tokens if num_total_tokens > 0 else 0
    
    # Most common tokens
    token_counts = Counter(tokens)
    most_common_tokens = token_counts.most_common(num_tokens)
    
    stats = {
        "Total tokens": num_total_tokens,
        "Unique tokens": num_unique_tokens,
        "Total characters": num_characters,
        "Lexical diversity": lexical_diversity,
        "Most common tokens": most_common_tokens
    }
    
    if verbose:
        for key, value in stats.items():
            print(f"{key}: {value}")
    
    return stats

# Ensure 'tokens' column exists before running stats
if 'tokens' not in lyrics_data.columns:
    print("Tokens column not found. Check your pipeline or input data.")
else:
    # Calculate descriptive statistics for Cher's lyrics
    cher_tokens = [token for sublist in lyrics_data[lyrics_data['artist'] == 'cher']['tokens'] for token in sublist]
    cher_stats = descriptive_stats(cher_tokens, num_tokens=10, verbose=False)
    print("Cher's Lyrics Statistics:")
    for key, value in cher_stats.items():
        print(f"{key}: {value}")

    # Calculate descriptive statistics for Robyn's lyrics
    robyn_tokens = [token for sublist in lyrics_data[lyrics_data['artist'] == 'robyn']['tokens'] for token in sublist]
    robyn_stats = descriptive_stats(robyn_tokens, num_tokens=10, verbose=False)
    print("Robyn's Lyrics Statistics:")
    for key, value in robyn_stats.items():
        print(f"{key}: {value}")


Tokens column not found. Check your pipeline or input data.


Q: What are some observations about the top tokens? Do you notice any interesting items on the list? 

A: My code did not return any top tokens.

## Build word clouds for all four corpora. 

For building wordclouds, we'll follow exactly the code of the text. The code in this section can be found [here](https://github.com/blueprints-for-text-analytics-python/blueprints-text/blob/master/ch01/First_Insights.ipynb). If you haven't already, you should absolutely clone the repository that accompanies the book. 


In [60]:
from matplotlib import pyplot as plt

def wordcloud(word_freq, title=None, max_words=200, stopwords=None):

    wc = WordCloud(width=800, height=400, 
                   background_color= "black", colormap="Paired", 
                   max_font_size=150, max_words=max_words)
    
    # convert data frame into dict
    if type(word_freq) == pd.Series:
        counter = Counter(word_freq.fillna(0).to_dict())
    else:
        counter = word_freq

    # filter stop words in frequency counter
    if stopwords is not None:
        counter = {token:freq for (token, freq) in counter.items() 
                              if token not in stopwords}
    wc.generate_from_frequencies(counter)
 
    plt.title(title) 

    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    
    
def count_words(df, column='tokens', preprocess=None, min_freq=2):

    # process tokens and update counter
    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

    # create counter and run through all data
    counter = Counter()
    df[column].map(update)

    # transform counter into data frame
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq')
    freq_df.index.name = 'token'
    
    return freq_df.sort_values('freq', ascending=False)

# Generate word frequencies for Cher and Robyn
cher_word_freq = count_words(lyrics_data[lyrics_data['artist'] == 'cher'], column='tokens', min_freq=5)
robyn_word_freq = count_words(lyrics_data[lyrics_data['artist'] == 'robyn'], column='tokens', min_freq=5)

# Generate word clouds for Cher and Robyn
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
wordcloud(cher_word_freq['freq'], title="Cher's Lyrics")

plt.subplot(1, 2, 2)
wordcloud(robyn_word_freq['freq'], title="Robyn's Lyrics")

plt.tight_layout()
plt.show()

KeyError: 'tokens'

Q: What observations do you have about these (relatively straightforward) wordclouds? 

A: The words that appear more frequently are bigger and more visible while the lesser used words are smaller to depict their lower frequency of usage.